In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install holidays

In [ ]:
!pip install onnxruntime-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.2/283.2 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
import holidays

In [ ]:
def get_day_of_week():
    date_obj = datetime.strptime(datetime.today().strftime('%m/%d/%Y'), '%m/%d/%Y')
    day = date_obj.strftime('%A')
    return day

def is_holiday(date, country='US'):
    us_holidays = holidays.CountryHoliday(country)
    return date in us_holidays

In [ ]:
print(get_day_of_week())

Monday


In [ ]:
import requests
from bs4 import BeautifulSoup
import re
url = 'https://www.timeanddate.com/weather/sri-lanka/colombo'

In [ ]:
content = requests.get(url=url)
soup = BeautifulSoup(content.content, features="html.parser")
lists = soup.find_all('table', attrs={'class': "table table--left table--inner-borders-rows"})
print(lists)

[<table class="table table--left table--inner-borders-rows"><tbody><tr><th>Location: </th><td>Colombo</td></tr><tr><th>Current Time: </th><td id="wtct">8 Jul 2025, 00:38:42</td></tr><tr><th>Latest Report: </th><td>7 Jul 2025, 23:30</td></tr><tr><th>Visibility: </th><td>20 km</td></tr><tr><th>Pressure: </th><td>1011 mbar</td></tr><tr><th>Humidity: </th><td>73%</td></tr><tr><th>Dew Point: </th><td>23 °C</td></tr></tbody></table>]


In [ ]:
soup = BeautifulSoup(str(lists[0]), "html.parser")
data = {}

for row in soup.find_all("tr"):
    th = row.find("th").text.strip().replace(":", "")
    td = row.find("td").text.strip()
    data[th] = td

print(data)

{'Location': 'Colombo', 'Current Time': '8 Jul 2025, 00:38:42', 'Latest Report': '7 Jul 2025, 23:30', 'Visibility': '20\xa0km', 'Pressure': '1011 mbar', 'Humidity': '73%', 'Dew Point': '23\xa0°C'}


In [ ]:
import requests

API_KEY = "5b7998388e713d3b50a88ea385980bd9"
CITY = "Berlin"
URL = f"http://api.openweathermap.org/data/2.5/weather?q={CITY}&appid={API_KEY}&units=metric"

response = requests.get(URL)
data = response.json()

if response.status_code == 200:
    print(f"City: {data['name']}")
    print(f"Temperature: {data['main']['temp']}°C")
    print(f"Weather: {data['weather'][0]['description']}")
else:
    print("Error:", data["message"])


City: Berlin
Temperature: 20.06°C
Weather: overcast clouds


In [ ]:
import requests
import time

geo_url = f"http://api.openweathermap.org/data/2.5/weather?q={CITY}&appid={API_KEY}"
geo_response = requests.get(geo_url).json()
lat = geo_response["coord"]["lat"]
lon = geo_response["coord"]["lon"]

date = "2024-02-20"
timestamp = int(time.mktime(time.strptime(date, "%Y-%m-%d")))
history_url = f"https://api.openweathermap.org/data/3.0/onecall/timemachine?lat={lat}&lon={lon}&dt={timestamp}&appid={API_KEY}&units=metric"
response = requests.get(history_url)
data = response.json()
if response.status_code == 200:
    print(f"City: {CITY}")
    print(f"Date: {date}")
    print(f"Temperature: {data['data'][0]['temp']}°C")
    print(f"Weather: {data['data'][0]['weather'][0]['description']}")
else:
    print("Error:", data.get("message", "Could not fetch data"))


Error: Please note that using One Call 3.0 requires a separate subscription to the One Call by Call plan. Learn more here https://openweathermap.org/price. If you have a valid subscription to the One Call by Call plan, but still receive this error, then please see https://openweathermap.org/faq#error401 for more info.


In [ ]:
def get_climate_conditions(date):
  geo_url = f"http://api.openweathermap.org/data/2.5/weather?q={CITY}&appid={API_KEY}"
  geo_response = requests.get(geo_url).json()
  lat = geo_response["coord"]["lat"]
  lon = geo_response["coord"]["lon"]
  # date = "2024-02-20"
  timestamp = int(time.mktime(time.strptime(date, "%Y-%m-%d")))
  history_url = f"https://api.openweathermap.org/data/3.0/onecall/timemachine?lat={lat}&lon={lon}&dt={timestamp}&appid={API_KEY}&units=metric"
  response = requests.get(history_url)
  data = response.json()
  if response.status_code == 200:
      print(f"City: {CITY}")
      print(f"Date: {date}")
      print(f"Temperature: {data['data'][0]['temp']}°C")
      print(f"Weather: {data['data'][0]['weather'][0]['description']}")
      return data['data'][0]['temp'] , data['data'][0]['weather'][0]['description']
  else:
      URL = f"http://api.openweathermap.org/data/2.5/weather?q={CITY}&appid={API_KEY}&units=metric"
      response = requests.get(URL)
      data = response.json()
      if response.status_code == 200:
          print(f"City: {data['name']}")
          print(f"Temperature: {data['main']['temp']}°C")
          print(f"Weather: {data['weather'][0]['description']}")
          return data['main']['temp'] , data['weather'][0]['description']
      else:
          print("Error:", data["message"])

In [ ]:
get_climate_conditions("2024-02-20")

City: Berlin
Temperature: 20.06°C
Weather: overcast clouds


(20.06, 'overcast clouds')

In [ ]:
main_path = '/content/drive/MyDrive/projects_data/FYP_20_Erandi'

In [ ]:
df_climates_german = pd.read_csv(f"{main_path}/german_climate_data.csv")
df_climates_german.head(5)

,dt,dt_iso,timezone,city_name,lat,lon,temp,visibility,dew_point,feels_like,...,wind_gust,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_id,weather_main,weather_description,weather_icon
0,283996800,1979-01-01 00:00:00 +0000 UTC,3600,Germany,51.165691,10.451526,254.01,NaN,252.07,247.01,...,NaN,NaN,NaN,0.16,NaN,100,600,Snow,light snow,13n
1,284000400,1979-01-01 01:00:00 +0000 UTC,3600,Germany,51.165691,10.451526,254.34,NaN,252.40,247.34,...,NaN,NaN,NaN,0.15,NaN,100,600,Snow,light snow,13n
2,284004000,1979-01-01 02:00:00 +0000 UTC,3600,Germany,51.165691,10.451526,254.70,NaN,252.62,247.70,...,NaN,NaN,NaN,0.20,NaN,100,600,Snow,light snow,13n
3,284007600,1979-01-01 03:00:00 +0000 UTC,3600,Germany,51.165691,10.451526,254.46,NaN,252.00,247.46,...,NaN,NaN,NaN,0.30,NaN,99,600,Snow,light snow,13n
4,284011200,1979-01-01 04:00:00 +0000 UTC,3600,Germany,51.165691,10.451526,254.76,NaN,252.30,247.76,...,NaN,NaN,NaN,0.28,NaN,93,600,Snow,light snow,13n


In [ ]:
# Remove the " UTC" suffix
df_climates_german['dt_iso'] = df_climates_german['dt_iso'].str.replace(' UTC', '', regex=False)

# Parse the cleaned datetime string with format specified
df_climates_german['dt_iso'] = pd.to_datetime(df_climates_german['dt_iso'], utc=True)


In [ ]:
# Filter after 2020-01-01
df_climates_german = df_climates_german[df_climates_german['dt_iso'] >= pd.Timestamp('2020-01-01', tz='UTC')]

# Reformat to 'YYYY-MM-DD' string
df_climates_german['dt_iso'] = df_climates_german['dt_iso'].dt.strftime('%Y-%m-%d')


In [ ]:
df_climates_german.head()

,dt,dt_iso,timezone,city_name,lat,lon,temp,visibility,dew_point,feels_like,...,wind_gust,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_id,weather_main,weather_description,weather_icon
359400,1577836800,2020-01-01,3600,Germany,51.165691,10.451526,275.18,NaN,274.32,273.61,...,NaN,NaN,NaN,NaN,NaN,77,803,Clouds,broken clouds,04n
359401,1577840400,2020-01-01,3600,Germany,51.165691,10.451526,274.31,NaN,273.45,272.28,...,NaN,NaN,NaN,NaN,NaN,77,803,Clouds,broken clouds,04n
359402,1577844000,2020-01-01,3600,Germany,51.165691,10.451526,273.82,NaN,273.12,271.94,...,NaN,NaN,NaN,NaN,NaN,78,803,Clouds,broken clouds,04n
359403,1577847600,2020-01-01,3600,Germany,51.165691,10.451526,272.80,NaN,272.18,271.20,...,NaN,NaN,NaN,NaN,NaN,81,803,Clouds,broken clouds,04n
359404,1577851200,2020-01-01,3600,Germany,51.165691,10.451526,272.52,NaN,272.03,272.52,...,NaN,NaN,NaN,NaN,NaN,79,803,Clouds,broken clouds,04n


In [ ]:
df_climates_german['dt_iso'] = pd.to_datetime(df_climates_german['dt_iso'])
df_climates_german['month'] = df_climates_german['dt_iso'].dt.strftime('%b')
df_climates_german['year'] = df_climates_german['dt_iso'].dt.year


In [ ]:
df = pd.read_csv(f"{main_path}/synthetic_beverage_sales_data.csv")
df.head(5)

,Order_ID,Customer_ID,Customer_Type,Product,Category,Unit_Price,Quantity,Discount,Total_Price,Region,Order_Date
0,ORD1,CUS1496,B2B,Vio Wasser,Water,1.66,53,0.10,79.18,Baden-Württemberg,2023-08-23
1,ORD1,CUS1496,B2B,Evian,Water,1.56,90,0.10,126.36,Baden-Württemberg,2023-08-23
2,ORD1,CUS1496,B2B,Sprite,Soft Drinks,1.17,73,0.05,81.14,Baden-Württemberg,2023-08-23
3,ORD1,CUS1496,B2B,Rauch Multivitamin,Juices,3.22,59,0.10,170.98,Baden-Württemberg,2023-08-23
4,ORD1,CUS1496,B2B,Gerolsteiner,Water,0.87,35,0.10,27.40,Baden-Württemberg,2023-08-23


In [ ]:
df = df.drop(columns=['Region' , 'Order_ID' , 'Customer_Type' , 'Customer_ID'])
df.head(5)

,Product,Category,Unit_Price,Quantity,Discount,Total_Price,Order_Date
0,Vio Wasser,Water,1.66,53,0.10,79.18,2023-08-23
1,Evian,Water,1.56,90,0.10,126.36,2023-08-23
2,Sprite,Soft Drinks,1.17,73,0.05,81.14,2023-08-23
3,Rauch Multivitamin,Juices,3.22,59,0.10,170.98,2023-08-23
4,Gerolsteiner,Water,0.87,35,0.10,27.40,2023-08-23


In [ ]:
import pandas as pd

# Example: if your Order_Date column is not already datetime
df['Order_Date'] = pd.to_datetime(df['Order_Date'])

# Get the minimum and maximum date
min_date = df['Order_Date'].min()
max_date = df['Order_Date'].max()

print("Min Date:", min_date)
print("Max Date:", max_date)


Min Date: 2021-01-01 00:00:00
Max Date: 2023-12-30 00:00:00


In [ ]:
df = df[df['Category'] == "Juices"]
df.head()

,Product,Category,Unit_Price,Quantity,Discount,Total_Price,Order_Date
3,Rauch Multivitamin,Juices,3.22,59,0.10,170.98,2023-08-23
6,Tomato Juice,Juices,2.14,44,0.10,84.74,2022-11-20
11,Hohes C Orange,Juices,1.52,8,0.05,11.55,2023-08-05
16,Mango Juice,Juices,2.50,10,0.05,23.75,2023-02-18
21,Rauch Multivitamin,Juices,2.31,7,0.00,16.17,2021-01-12


In [ ]:
# Remove duplicate rows based on the 'date' column (keep first occurrence)
df_unique = df_climates_german.drop_duplicates(subset='dt_iso', keep='first')


In [ ]:
# Suppose your DataFrame is named df and you want rows where column 'city' is 'Berlin'
filtered_rows = df_unique[df_unique['dt_iso'] == '2023-08-23']


In [ ]:
filtered_rows.head()

,dt,dt_iso,timezone,city_name,lat,lon,temp,visibility,dew_point,feels_like,...,rain_3h,snow_1h,snow_3h,clouds_all,weather_id,weather_main,weather_description,weather_icon,month,year
391320,1692748800,2023-08-23,7200,Germany,51.165691,10.451526,289.08,NaN,287.95,289.16,...,NaN,NaN,NaN,0,800,Clear,sky is clear,01n,Aug,2023


In [ ]:
filtered_rows.shape

(1, 30)

In [ ]:
df_unique['date'] = pd.to_datetime(df_unique['dt_iso']).dt.strftime('%Y-%m-%d')
df['date'] = pd.to_datetime(df['Order_Date']).dt.strftime('%Y-%m-%d')
merged_df = pd.merge(df_unique, df, on='date', how='inner')  # or 'outer' / 'left' / 'right'


/tmp/ipython-input-111-230958505.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique['date'] = pd.to_datetime(df_unique['dt_iso']).dt.strftime('%Y-%m-%d')


In [ ]:
merged_df.head()

,dt,dt_iso,timezone,city_name,lat,lon,temp,visibility,dew_point,feels_like,...,month,year,date,Product,Category,Unit_Price,Quantity,Discount,Total_Price,Order_Date
0,1609459200,2021-01-01,3600,Germany,51.165691,10.451526,274.84,NaN,274.13,273.16,...,Jan,2021,2021-01-01,Hohes C Orange,Juices,1.75,3,0.0,5.25,2021-01-01
1,1609459200,2021-01-01,3600,Germany,51.165691,10.451526,274.84,NaN,274.13,273.16,...,Jan,2021,2021-01-01,Granini Apple,Juices,1.59,3,0.0,4.77,2021-01-01
2,1609459200,2021-01-01,3600,Germany,51.165691,10.451526,274.84,NaN,274.13,273.16,...,Jan,2021,2021-01-01,Passion Fruit Juice,Juices,2.93,12,0.0,35.16,2021-01-01
3,1609459200,2021-01-01,3600,Germany,51.165691,10.451526,274.84,NaN,274.13,273.16,...,Jan,2021,2021-01-01,Mango Juice,Juices,3.12,7,0.0,21.84,2021-01-01
4,1609459200,2021-01-01,3600,Germany,51.165691,10.451526,274.84,NaN,274.13,273.16,...,Jan,2021,2021-01-01,Rauch Multivitamin,Juices,2.31,4,0.0,9.24,2021-01-01


In [ ]:
agg_dict = {col: 'first' for col in merged_df.columns if col not in ['Quantity']}
agg_dict['Quantity'] = 'sum'

df_grouped = merged_df.groupby(['date', 'Product'], as_index=False).agg(agg_dict)


In [ ]:
df_grouped.head()

,dt,dt_iso,timezone,city_name,lat,lon,temp,visibility,dew_point,feels_like,...,month,year,date,Product,Category,Unit_Price,Discount,Total_Price,Order_Date,Quantity
0,1609459200,2021-01-01,3600,Germany,51.165691,10.451526,274.84,NaN,274.13,273.16,...,Jan,2021,2021-01-01,Cranberry Juice,Juices,3.46,0.0,38.06,2021-01-01,5610
1,1609459200,2021-01-01,3600,Germany,51.165691,10.451526,274.84,NaN,274.13,273.16,...,Jan,2021,2021-01-01,Granini Apple,Juices,1.59,0.0,4.77,2021-01-01,7471
2,1609459200,2021-01-01,3600,Germany,51.165691,10.451526,274.84,NaN,274.13,273.16,...,Jan,2021,2021-01-01,Hohes C Orange,Juices,1.75,0.0,5.25,2021-01-01,6432
3,1609459200,2021-01-01,3600,Germany,51.165691,10.451526,274.84,NaN,274.13,273.16,...,Jan,2021,2021-01-01,Mango Juice,Juices,3.12,0.0,21.84,2021-01-01,6208
4,1609459200,2021-01-01,3600,Germany,51.165691,10.451526,274.84,NaN,274.13,273.16,...,Jan,2021,2021-01-01,Passion Fruit Juice,Juices,2.93,0.0,35.16,2021-01-01,6493


In [ ]:
# Step 1: Check null value percentage
null_percent = df_grouped.isnull().mean() * 100
print(null_percent)


dt                       0.000000
dt_iso                   0.000000
timezone                 0.000000
city_name                0.000000
lat                      0.000000
lon                      0.000000
temp                     0.000000
visibility             100.000000
dew_point                0.000000
feels_like               0.000000
temp_min                 0.000000
temp_max                 0.000000
pressure                 0.000000
sea_level              100.000000
grnd_level             100.000000
humidity                 0.000000
wind_speed               0.000000
wind_deg                 0.000000
wind_gust              100.000000
rain_1h                 87.477148
rain_3h                100.000000
snow_1h                 98.263254
snow_3h                100.000000
clouds_all               0.000000
weather_id               0.000000
weather_main             0.000000
weather_description      0.000000
weather_icon             0.000000
month                    0.000000
year          

In [ ]:
# Keep only columns with <= 70% nulls
df_cleaned = df_grouped.loc[:, null_percent <= 70]


In [ ]:
df_cleaned.head()

,dt,dt_iso,timezone,city_name,lat,lon,temp,dew_point,feels_like,temp_min,...,month,year,date,Product,Category,Unit_Price,Discount,Total_Price,Order_Date,Quantity
0,1609459200,2021-01-01,3600,Germany,51.165691,10.451526,274.84,274.13,273.16,274.18,...,Jan,2021,2021-01-01,Cranberry Juice,Juices,3.46,0.0,38.06,2021-01-01,5610
1,1609459200,2021-01-01,3600,Germany,51.165691,10.451526,274.84,274.13,273.16,274.18,...,Jan,2021,2021-01-01,Granini Apple,Juices,1.59,0.0,4.77,2021-01-01,7471
2,1609459200,2021-01-01,3600,Germany,51.165691,10.451526,274.84,274.13,273.16,274.18,...,Jan,2021,2021-01-01,Hohes C Orange,Juices,1.75,0.0,5.25,2021-01-01,6432
3,1609459200,2021-01-01,3600,Germany,51.165691,10.451526,274.84,274.13,273.16,274.18,...,Jan,2021,2021-01-01,Mango Juice,Juices,3.12,0.0,21.84,2021-01-01,6208
4,1609459200,2021-01-01,3600,Germany,51.165691,10.451526,274.84,274.13,273.16,274.18,...,Jan,2021,2021-01-01,Passion Fruit Juice,Juices,2.93,0.0,35.16,2021-01-01,6493


In [ ]:
for col in df_cleaned.columns:
    print(f"\nColumn: {col}")
    print(df_cleaned[col].unique())



Column: dt
[1609459200 1609545600 1609632000 ... 1703721600 1703808000 1703894400]

Column: dt_iso
<DatetimeArray>
['2021-01-01 00:00:00', '2021-01-02 00:00:00', '2021-01-03 00:00:00',
 '2021-01-04 00:00:00', '2021-01-05 00:00:00', '2021-01-06 00:00:00',
 '2021-01-07 00:00:00', '2021-01-08 00:00:00', '2021-01-09 00:00:00',
 '2021-01-10 00:00:00',
 ...
 '2023-12-21 00:00:00', '2023-12-22 00:00:00', '2023-12-23 00:00:00',
 '2023-12-24 00:00:00', '2023-12-25 00:00:00', '2023-12-26 00:00:00',
 '2023-12-27 00:00:00', '2023-12-28 00:00:00', '2023-12-29 00:00:00',
 '2023-12-30 00:00:00']
Length: 1094, dtype: datetime64[ns]

Column: timezone
[3600 7200]

Column: city_name
['Germany']

Column: lat
[51.165691]

Column: lon
[10.451526]

Column: temp
[274.84 273.85 274.9  273.74 274.29 274.81 272.97 274.17 273.28 271.28
 268.91 274.65 274.66 273.54 271.85 270.41 271.01 272.8  276.19 276.81
 279.52 282.29 274.37 273.64 273.26 271.62 275.18 274.19 279.26 272.2
 271.86 276.01 281.87 280.77 276.04 27

In [ ]:
drop_columns = ['dt' , 'dt_iso' , 'city_name','timezone','lat','lon','date','Category','Order_Date','weather_id','weather_icon']
df_dropped = df_cleaned.drop(columns=drop_columns)
df_dropped.shape

(7658, 19)

In [ ]:
print(df_dropped.columns.tolist())


['temp', 'dew_point', 'feels_like', 'temp_min', 'temp_max', 'pressure', 'humidity', 'wind_speed', 'wind_deg', 'clouds_all', 'weather_main', 'weather_description', 'month', 'year', 'Product', 'Unit_Price', 'Discount', 'Total_Price', 'Quantity']


In [ ]:
df_cpi = pd.read_csv(f"{main_path}/german_cpi_data.csv")
df_cpi.head()

,Unnamed: 0,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Annual
0,0,2025,120.3,120.8,121.2,121.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2024,117.6,118.1,118.6,119.2,119.3,119.4,119.8,119.7,119.7,120.2,119.9,120.5,119.3
2,2,2023,114.3,115.2,116.1,116.6,116.5,116.8,117.1,117.5,117.8,117.8,117.3,117.4,116.7
3,3,2022,105.2,106.0,108.1,108.8,109.8,109.8,110.3,110.7,112.7,113.5,113.7,113.2,110.2
4,4,2021,101.0,101.6,102.1,102.4,102.6,102.9,103.4,103.5,103.8,104.3,104.5,104.7,103.1


In [ ]:
jan_value = df_cpi.loc[df_cpi['Year'] == 2024, 'Jan'].iloc[0]
print(jan_value)


117.6


In [ ]:
df_dropped['cpi'] = 0

In [ ]:
def get_cpi_data(row):
    year = row['year']
    month = row['month']
    cpi_value = df_cpi.loc[(df_cpi['Year'] == int(year)), month].iloc[0]
    row['cpi'] = cpi_value
    return row

In [ ]:
df_dropped = df_dropped.apply(get_cpi_data, axis=1)

In [ ]:
df_dropped.head()

,temp,dew_point,feels_like,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,clouds_all,weather_main,weather_description,month,year,Product,Unit_Price,Discount,Total_Price,Quantity,cpi
0,274.84,274.13,273.16,274.18,275.89,1005,95,1.61,184,75,Snow,light snow,Jan,2021,Cranberry Juice,3.46,0.0,38.06,5610,101.0
1,274.84,274.13,273.16,274.18,275.89,1005,95,1.61,184,75,Snow,light snow,Jan,2021,Granini Apple,1.59,0.0,4.77,7471,101.0
2,274.84,274.13,273.16,274.18,275.89,1005,95,1.61,184,75,Snow,light snow,Jan,2021,Hohes C Orange,1.75,0.0,5.25,6432,101.0
3,274.84,274.13,273.16,274.18,275.89,1005,95,1.61,184,75,Snow,light snow,Jan,2021,Mango Juice,3.12,0.0,21.84,6208,101.0
4,274.84,274.13,273.16,274.18,275.89,1005,95,1.61,184,75,Snow,light snow,Jan,2021,Passion Fruit Juice,2.93,0.0,35.16,6493,101.0


In [ ]:
drop_columns = ['year' , 'month']
df_dropped = df_dropped.drop(columns=drop_columns)
df_dropped.shape

(7658, 18)

In [ ]:
non_numeric_columns = ['weather_main','weather_description','Product']
for non_numeric in non_numeric_columns:
  unique_values = df_dropped[non_numeric].unique()
  print(non_numeric ,len(list(unique_values)) )
  classes = {}
  for value in list(unique_values):
    classes[value] = list(unique_values).index(value)
  print(classes)
  df_dropped[non_numeric] = df_dropped[non_numeric].map(classes)
df_dropped.head(5)

weather_main 4
{'Snow': 0, 'Clouds': 1, 'Rain': 2, 'Clear': 3}
weather_description 9
{'light snow': 0, 'broken clouds': 1, 'overcast clouds': 2, 'scattered clouds': 3, 'few clouds': 4, 'light rain': 5, 'snow': 6, 'sky is clear': 7, 'moderate rain': 8}
Product 7
{'Cranberry Juice': 0, 'Granini Apple': 1, 'Hohes C Orange': 2, 'Mango Juice': 3, 'Passion Fruit Juice': 4, 'Rauch Multivitamin': 5, 'Tomato Juice': 6}


,temp,dew_point,feels_like,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,clouds_all,weather_main,weather_description,Product,Unit_Price,Discount,Total_Price,Quantity,cpi
0,274.84,274.13,273.16,274.18,275.89,1005,95,1.61,184,75,0,0,0,3.46,0.0,38.06,5610,101.0
1,274.84,274.13,273.16,274.18,275.89,1005,95,1.61,184,75,0,0,1,1.59,0.0,4.77,7471,101.0
2,274.84,274.13,273.16,274.18,275.89,1005,95,1.61,184,75,0,0,2,1.75,0.0,5.25,6432,101.0
3,274.84,274.13,273.16,274.18,275.89,1005,95,1.61,184,75,0,0,3,3.12,0.0,21.84,6208,101.0
4,274.84,274.13,273.16,274.18,275.89,1005,95,1.61,184,75,0,0,4,2.93,0.0,35.16,6493,101.0


In [ ]:
df_dropped.to_csv(f"{main_path}/synthetic_beverage_sales_enhanced_climate_cpi.csv", index=False)

In [ ]:
def model_train_evaluate(df_):
  target_column_name = 'Quantity'
  if target_column_name in df_.columns:
    y = df_[target_column_name]
    df_ = df_.drop(columns=[target_column_name])
    print(y)
    X = pd.get_dummies(df_)
    print(X)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = RandomForestClassifier(n_estimators=100, random_state=42)  # You can adjust hyperparameters as needed
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(y_test[0:20])
    print(y_pred[0:20])
    accuracy = accuracy_score(y_test, y_pred)
    print(f'Accuracy: {accuracy:.2f}')
  else:
    print(f"Column '{target_column_name}' not found in the dataset. Please check the column name's case sensitivity.")

In [ ]:
model_train_evaluate(df_dropped)

0       5610
1       7471
2       6432
3       6208
4       6493
        ... 
7653    6421
7654    6144
7655    6030
7656    7743
7657    6678
Name: Quantity, Length: 7658, dtype: int64
        temp  dew_point  feels_like  temp_min  temp_max  pressure  humidity  \
0     274.84     274.13      273.16    274.18    275.89      1005        95   
1     274.84     274.13      273.16    274.18    275.89      1005        95   
2     274.84     274.13      273.16    274.18    275.89      1005        95   
3     274.84     274.13      273.16    274.18    275.89      1005        95   
4     274.84     274.13      273.16    274.18    275.89      1005        95   
...      ...        ...         ...       ...       ...       ...       ...   
7653  281.11     278.40      277.60    280.86    282.16      1010        83   
7654  281.11     278.40      277.60    280.86    282.16      1010        83   
7655  281.11     278.40      277.60    280.86    282.16      1010        83   
7656  281.11     278.40 

In [ ]:
import requests
import pandas as pd

URL = "http://api.worldbank.org/v2/country/DE/indicator/FP.CPI.TOTL.ZG?format=json"
response = requests.get(URL)
data = response.json()
inflation_rate = data[1][0]['value']
year = data[1][0]['date']
print(f"Germany's Inflation Rate in {year}: {inflation_rate}%")


In [ ]:
!pip install tradingeconomics

In [ ]:
import tradingeconomics as te

In [ ]:
te.login('76821a21b89b4b6:lgenpsh2ejox0rw')

In [ ]:
te.getIndicatorData(indicators='inflation rate')